# Score multi-session events using the replay score from Davidson et al.

In [1]:
import numpy as np
import os
import pandas as pd
import warnings

import nelpy as nel

warnings.filterwarnings("ignore")

/opt/conda/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.


### Load experimental data from Google Cloud bucket

In [2]:
import gcsfs
import pandas as pd

fs = gcsfs.GCSFileSystem(project='polar-program-784', token='cloud')
print(fs.ls('kemerelab-data/diba'))

with fs.open('kemerelab-data/diba/gor01vvp01pin01-metadata.h5', 'rb') as fid:
    with pd.HDFStore('gor01vvp01pin01-metadata.h5', mode="r", driver="H5FD_CORE",
            driver_core_backing_store=0,
            driver_core_image=fid.read()
            ) as store:
        df = store['Session_Metadata']
        df2 = store['Subset_Metadata']


['kemerelab-data/diba/', 'kemerelab-data/diba/gor01vvp01-metadata.h5', 'kemerelab-data/diba/gor01vvp01_processed_speed.nel', 'kemerelab-data/diba/gor01vvp01pin01-metadata.h5', 'kemerelab-data/diba/gor01vvp01pin01_processed_speed.nel']


In [3]:
# %%timeit # beware - this will run it 7 times to get a time! 36 s for 1.4 GB file

with fs.open('kemerelab-data/diba/gor01vvp01pin01_processed_speed.nel', 'rb') as fid:
    jar = nel.load_pkl('',fileobj=fid) # currently requires a specific nelpy branch

exp_data = jar.exp_data
aux_data = jar.aux_data
#del jar


### Define subset of sessions to score

In [4]:
# restrict sessions to explore to a smaller subset
min_n_placecells = 20
min_n_PBEs = 30 # 27 total events ==> minimum 21 events in training set

df2_subset = df2[(df2.n_PBEs >= min_n_PBEs) & (df2.n_placecells >= min_n_placecells)]

sessions = df2_subset['time'].values.tolist()
segments = df2_subset['segment'].values.tolist()

print('Evaluating subset of {} sessions'.format(len(sessions)))

df2_subset.sort_values(by=['n_PBEs', 'n_placecells'], ascending=[0,0])

Evaluating subset of 18 sessions


,animal,month,day,time,track,segment,duration,n_cells,n_placecells,n_PBEs,Notes,prescreen_z
54,gor01,6,9,22-24-40,two,short,1620.0,203,61,301,NaN,NaN
1,gor01,6,7,16-40-19,two,short,1330.0,117,46,277,NaN,NaN
28,pin01,11,1,12-58-54,unknown,long,1670.0,49,21,238,NaN,NaN
29,pin01,11,1,12-58-54,unknown,short,925.0,49,26,222,NaN,NaN
0,gor01,6,7,16-40-19,two,long,1180.0,117,43,150,NaN,NaN
42,gor01,6,9,1-22-43,one,long,1012.0,203,62,117,NaN,NaN
53,gor01,6,9,22-24-40,two,long,912.0,203,66,103,NaN,NaN
43,gor01,6,9,1-22-43,one,short,617.0,203,71,91,NaN,NaN
18,gor01,6,8,21-16-25,two,long,720.0,171,82,57,NaN,NaN
60,vvp01,4,9,17-29-30,one,short,490.0,68,32,46,NaN,NaN


### Parallel scoring

**NOTE:** it is relatively easy (syntax-wise) to score each session as a parallel task, but since the Bayesian scoring takes such a long time to compute, we can be more efficient (higher % utilization) by further parallelizing over events, and not just over sessions. This further level of parallelization makes the bookkeeping a little ugly, so I provide the code for both approaches here.

In [5]:
# It would be easy enough to set up the dask hooks for by_session, but right now it's only done for by_event.
# So don't change these or it won't work.
parallelize_by_session = False # won't work currently with Dask
parallelize_by_event = True

n_jobs = 20 # set this equal to number of cores (or number of workers if using the dask-distribued joblib backend)
n_shuffles = 2 # 5000
n_samples = 100 # 35000
w=3 # single sided bandwidth (0 means only include bin who's center is under line, 3 means a total of 7 bins)

In [6]:
from dask.distributed import Client
# client = Client('dask-scheduler:8786') # fill in value from dask-kubernetes startup if connecting remotely
client = Client() # Use this to create a local scheduluer
client

Client Scheduler: tcp://127.0.0.1:33341 Dashboard: http://127.0.0.1:8787,Cluster Workers: 16 Cores: 16 Memory: 63.34 GB


In [7]:
# Not set up for dask!!!

if parallelize_by_session:
    from joblib import Parallel, delayed 


    # A function that can be called to do work:
    def work_sessions(arg):    

        # Split the list to individual variables:
        ii, bst, tc = arg    

        scores, shuffled_scores, percentiles = nel.analysis.replay.score_Davidson_final_bst_fast(bst=bst,
                                                                                            tuningcurve=tc,
                                                                                            w=w,
                                                                                            n_shuffles=n_shuffles,
                                                                                            n_samples=n_samples)

        return (ii, scores, shuffled_scores, percentiles)

    # List of instances to pass to work():
    parallel_sessions = [(ii, aux_data[session][segment]['PBEs'], aux_data[session][segment]['tc']) for (ii, (session, segment)) in enumerate(zip(sessions, segments))]

    parallel_results = Parallel(n_jobs=n_jobs, verbose=51)(map(delayed(work_sessions), parallel_sessions))

    # standardize parallel results
    idx = [result[0] for result in parallel_results]

    # check that parallel results came back IN ORDER:
    if nel.utils.is_sorted(idx):
        print('parallel results are ordered...')
    else:
        raise ValueError('results are not ordered! handle it here before proceeding...')

    scores_bayes = [result[1] for result in parallel_results]
    scores_bayes_shuffled = [result[2] for result in parallel_results]
    scores_bayes_percentile = [result[3] for result in parallel_results]

    results = dict()
    for ii, (session, segment) in enumerate(zip(sessions, segments)):
        try:
            results[session][segment] = dict()
        except KeyError:
            results[session] = dict()    
            results[session][segment] = dict()
        results[session][segment]['scores_bayes'] = scores_bayes[ii]
        results[session][segment]['scores_bayes_shuffled'] = scores_bayes_shuffled[ii]
        
        results[session][segment]['scores_bayes_percentile'] = scores_bayes_percentile[ii]

    print('done packing results')

In [9]:
# Set up for Dask
if parallelize_by_event:
    import time
    t0 = time.time()
    
    #from joblib import Parallel, delayed 
    import distributed.joblib
    from joblib import Parallel, parallel_backend, delayed

    # A function that can be called to do work:
    def work_events(arg):    
        # Split the list to individual variables:
        session, segment, ii, bst, tc = arg
        scores, shuffled_scores, percentiles = nel.analysis.replay.score_Davidson_final_bst_fast(bst=bst,
                                                                                            tuningcurve=tc,
                                                                                            w=w,
                                                                                            n_shuffles=n_shuffles,
                                                                                            n_samples=n_samples)

        t01 = time.time()
        return (session, segment, ii, scores, shuffled_scores, percentiles)

    # List of instances to pass to work():
    # unroll all events:
    parallel_events = []
    for session, segment in zip(sessions, segments):
        for nn in range(aux_data[session][segment]['PBEs'].n_epochs):
            parallel_events.append((session, segment, nn, aux_data[session][segment]['PBEs'][nn], aux_data[session][segment]['tc']))

# Option 1: Just use joblib - you need the "from joblib import Parallel, delayed" uncommented. 
#           (and not distributed.joblib?)
    #parallel_results = Parallel(n_jobs=n_jobs, verbose=51)(map(delayed(work_events), parallel_events))
    
# Option 2: Use dask-distrubuted for the joblib backend
#     NOTE fill in the scheduler ip returned from looking at "client" in the cell above
    with parallel_backend('dask.distributed', scheduler_host='tcp://127.0.0.1:33341'):
        parallel_results = Parallel(n_jobs=n_jobs, verbose=51)(map(delayed(work_events), parallel_events))

# Option 3: Use the dask-distributed client commands
    # parallel_results_future = client.map(work_events, parallel_events)
    # parallel_results = client.gather(parallel_results_future)
    
# Option 4: Use deep-dask parallelism
    # To be implemented. The key, apparently, in wrapping the "score_" function with delayed is to use the
    # "traverse=False" option. See https://github.com/dask/dask/issues/2785
    
    # standardize parallel results
    bdries_ = [aux_data[session][segment]['PBEs'].n_epochs for session, segment in zip(sessions, segments) ]
    bdries = np.cumsum(np.insert(bdries_,0,0))
    bdries

    sessions_ = np.array([result[0] for result in parallel_results])
    segments_ = np.array([result[1] for result in parallel_results])
    idx = [result[2] for result in parallel_results]

    scores_bayes_evt = np.array([float(result[3]) for result in parallel_results])
    scores_bayes_shuffled_evt = np.array([result[4].squeeze() for result in parallel_results])
    scores_bayes_percentile_evt = np.array([float(result[5]) for result in parallel_results])

    results = {}
    for nn in range(len(bdries)-1):
        session = np.unique(sessions_[bdries[nn]:bdries[nn+1]])
        if len(session) > 1:
            raise ValueError("parallel results in different format / order than expected!")
        session = session[0]
        segment = np.unique(segments_[bdries[nn]:bdries[nn+1]])
        if len(segment) > 1:
            raise ValueError("parallel results in different format / order than expected!")
        segment = segment[0]
        try:
            results[session][segment]['scores_bayes'] = scores_bayes_evt[bdries[nn]:bdries[nn+1]]
        except KeyError:
            try:
                results[session][segment] = dict()
                results[session][segment]['scores_bayes'] = scores_bayes_evt[bdries[nn]:bdries[nn+1]]
            except KeyError:
                results[session] = dict()
                results[session][segment] = dict()
                results[session][segment]['scores_bayes'] = scores_bayes_evt[bdries[nn]:bdries[nn+1]]

        results[session][segment]['scores_bayes_shuffled'] = scores_bayes_shuffled_evt[bdries[nn]:bdries[nn+1]]
        results[session][segment]['scores_bayes_percentile'] = scores_bayes_percentile_evt[bdries[nn]:bdries[nn+1]]

    print('done packing results')

t1 = time.time()    
print('Elapsed time ', t1 - t0)

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done   3 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done   4 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done   5 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done   6 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done   7 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done  11 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done  12 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done  13 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done  14 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done  15 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done  16 tasks      | elapsed:    0.9s
[Paralle

[Parallel(n_jobs=20)]: Done 134 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 135 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 136 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 137 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 138 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 139 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 140 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 142 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 143 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 144 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 145 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 146 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 147 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 148 tasks      | elapsed:    2.0s
[Parallel(n_jobs=20)]: Done 149 tasks      | elapsed:    2.0s
[Paralle

[Parallel(n_jobs=20)]: Done 280 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 281 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 282 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 283 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 284 tasks      | elapsed:    3.3s
[Parallel(n_jobs=20)]: Done 285 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 286 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 287 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 288 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 289 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 290 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 291 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 292 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 293 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 294 tasks      | elapsed:    3.4s
[Parallel(n_jobs=20)]: Done 295 tasks      | elapsed:    3.4s
[Paralle

[Parallel(n_jobs=20)]: Done 428 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 429 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 430 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 431 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 432 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 433 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 434 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 435 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 436 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 437 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 438 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 439 tasks      | elapsed:    5.0s
[Parallel(n_jobs=20)]: Done 440 tasks      | elapsed:    5.0s
[Parallel(n_jobs=20)]: Done 441 tasks      | elapsed:    5.1s
[Parallel(n_jobs=20)]: Done 442 tasks      | elapsed:    5.1s
[Parallel(n_jobs=20)]: Done 443 tasks      | elapsed:    5.1s
[Paralle

[Parallel(n_jobs=20)]: Done 572 tasks      | elapsed:    6.3s
[Parallel(n_jobs=20)]: Done 573 tasks      | elapsed:    6.3s
[Parallel(n_jobs=20)]: Done 574 tasks      | elapsed:    6.3s
[Parallel(n_jobs=20)]: Done 575 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 576 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 577 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 578 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 579 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 580 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 581 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 582 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 583 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 584 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 585 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 586 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 587 tasks      | elapsed:    6.4s
[Paralle

...

[Parallel(n_jobs=20)]: Done 1305 tasks      | elapsed:   12.6s
[Parallel(n_jobs=20)]: Done 1306 tasks      | elapsed:   12.6s
[Parallel(n_jobs=20)]: Done 1307 tasks      | elapsed:   12.6s
[Parallel(n_jobs=20)]: Done 1308 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1309 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1310 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1311 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1312 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1313 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1314 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1315 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1316 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1317 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1318 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1319 tasks      | elapsed:   12.7s
[Parallel(n_jobs=20)]: Done 1320 tasks      | elapsed: 

[Parallel(n_jobs=20)]: Done 1441 tasks      | elapsed:   14.1s
[Parallel(n_jobs=20)]: Done 1442 tasks      | elapsed:   14.1s
[Parallel(n_jobs=20)]: Done 1443 tasks      | elapsed:   14.1s
[Parallel(n_jobs=20)]: Done 1444 tasks      | elapsed:   14.1s
[Parallel(n_jobs=20)]: Done 1445 tasks      | elapsed:   14.1s
[Parallel(n_jobs=20)]: Done 1446 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done 1447 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done 1448 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done 1449 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done 1450 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done 1451 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done 1452 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done 1453 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done 1454 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done 1455 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done 1456 tasks      | elapsed: 

[Parallel(n_jobs=20)]: Done 1575 tasks      | elapsed:   15.5s
[Parallel(n_jobs=20)]: Done 1576 tasks      | elapsed:   15.5s
[Parallel(n_jobs=20)]: Done 1577 tasks      | elapsed:   15.5s
[Parallel(n_jobs=20)]: Done 1578 tasks      | elapsed:   15.5s
[Parallel(n_jobs=20)]: Done 1579 tasks      | elapsed:   15.5s
[Parallel(n_jobs=20)]: Done 1580 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 1581 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 1582 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 1583 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 1584 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 1585 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 1586 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 1587 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 1588 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 1589 tasks      | elapsed:   15.6s
[Parallel(n_jobs=20)]: Done 1590 tasks      | elapsed: 

[Parallel(n_jobs=20)]: Done 1714 tasks      | elapsed:   17.0s
[Parallel(n_jobs=20)]: Done 1715 tasks      | elapsed:   17.0s
[Parallel(n_jobs=20)]: Done 1716 tasks      | elapsed:   17.0s
[Parallel(n_jobs=20)]: Done 1717 tasks      | elapsed:   17.0s
[Parallel(n_jobs=20)]: Done 1718 tasks      | elapsed:   17.0s
[Parallel(n_jobs=20)]: Done 1719 tasks      | elapsed:   17.0s
[Parallel(n_jobs=20)]: Done 1720 tasks      | elapsed:   17.0s
[Parallel(n_jobs=20)]: Done 1721 tasks      | elapsed:   17.0s
[Parallel(n_jobs=20)]: Done 1722 tasks      | elapsed:   17.0s
[Parallel(n_jobs=20)]: Done 1723 tasks      | elapsed:   17.1s
[Parallel(n_jobs=20)]: Done 1724 tasks      | elapsed:   17.1s
[Parallel(n_jobs=20)]: Done 1725 tasks      | elapsed:   17.1s
[Parallel(n_jobs=20)]: Done 1726 tasks      | elapsed:   17.1s
[Parallel(n_jobs=20)]: Done 1727 tasks      | elapsed:   17.1s
[Parallel(n_jobs=20)]: Done 1728 tasks      | elapsed:   17.1s
[Parallel(n_jobs=20)]: Done 1729 tasks      | elapsed: 

[Parallel(n_jobs=20)]: Done 1883 out of 1883 | elapsed:   27.7s finished
done packing results
Elapsed time  29.060096263885498


### Save results to disk

In [ ]:
jar = nel.ResultsContainer(results=results, description='gor01 and vvp01 speed restricted results for best 20 candidate sessions')
jar.save_pkl('score_bayes_all_sessions.nel')